In [1]:
!pip install ipywidgets

import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import requests
from bs4 import BeautifulSoup
import os
import zipfile
from tqdm import tqdm
from pathlib import Path

import requests
import zipfile
import io
import os
import json

from collections import defaultdict
import toml

from proteobench.modules.quant.quant_lfq_ion_DDA import DDAQuantIonModule

In [2]:
# Examples to choose from:
#
# "https://github.com/Proteobench/Results_quant_ion_DDA/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_quant_peptidoform_DIA/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_subcellprofile_DOMLFQ_protein_DIA_EXPL/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_quant_peptidoform_DDA/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_quant_ion_DIA_diaPASEF/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_quant_ion_DIA_singlecell/archive/refs/heads/main.zip"
# "https://github.com/Proteobench/Results_quant_ion_DIA/archive/refs/heads/main.zip"

repo_url = "https://github.com/Proteobench/Results_quant_ion_DDA/archive/refs/heads/main.zip"

# Step 1: Download ZIP archive from GitHub
response = requests.get(repo_url)
zip_bytes = io.BytesIO(response.content)

# Step 2: Extract ZIP contents to a local folder
with zipfile.ZipFile(zip_bytes) as zip_ref:
    zip_ref.extractall("Results_quant_ion_DDA")

# Step 3: Prepare base directory
base_path = "Results_quant_ion_DDA/Results_quant_ion_DDA-main"

# Step 4: Initialize combined JSON container
combined_json = []

# Step 5: Walk through directory and read all JSON files
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)
                    combined_json.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error reading {file_path}: {e}")

# Step 6: Write combined JSON to a single output file (optional)
with open("combined_results.json", "w", encoding="utf-8") as out_file:
    json.dump(combined_json, out_file, ensure_ascii=False, indent=2)

print(f"Combined {len(combined_json)} JSON files into 'combined_results.json'.")

df = pd.json_normalize(combined_json)
df.head(5)

Combined 51 JSON files into 'combined_results.json'.


,id,old_new,software_name,software_version,search_engine,search_engine_version,ident_fdr_psm,ident_fdr_peptide,ident_fdr_protein,enable_match_between_runs,...,results.2.CV_q75,results.2.CV_q95,results.1.median_abs_epsilon,results.1.mean_abs_epsilon,results.1.variance_epsilon,results.1.nr_prec,results.1.CV_median,results.1.CV_q90,results.1.CV_q75,results.1.CV_q95
0,MaxQuant_20241216_122433,old,MaxQuant,2.5.1.0,Andromeda,None,None,0.01,0.01,False,...,0.291818,0.512293,0.199852,0.274547,0.160229,50302,0.202619,0.416519,0.291818,0.512293
1,MaxQuant_20241216_124040,old,MaxQuant,2.3.1.0,Andromeda,None,None,0.01,0.01,True,...,0.312472,0.569765,0.207123,0.304261,0.222469,50339,0.215507,0.460770,0.312472,0.569765
2,MaxQuant_20250506_151623,new,MaxQuant,2.3.1.0,Andromeda,NaN,0.01,None,0.01,True,...,0.312472,0.569765,0.207123,0.304261,0.222469,50339,0.215507,0.460770,0.312472,0.569765
3,FragPipe_20250504_164850,new,FragPipe,23.0,MSFragger,4.2,0.01,None,0.01,False,...,0.276832,0.560573,0.199952,0.309088,0.230707,57021,0.156473,0.437619,0.276832,0.560573
4,WOMBAT_20250506_073548,new,WOMBAT,0.9.11,Proline,nan,0.01,0.01,0.01,True,...,0.335215,0.643571,0.204178,0.329195,0.284523,61819,0.230370,0.507628,0.335215,0.643571


In [3]:
# Create a SelectMultiple widget with names as options
row_selector = widgets.SelectMultiple(
    options=[(f"{row['id']} (hash: {row['intermediate_hash']}, submission comments: {row['submission_comments']})", idx) for idx, row in df.iterrows()],
    description='Select Rows:',
    rows=10,  # Number of visible rows in the widget
    layout=widgets.Layout(width='50%')  # Adjust layout as needed
)

# Button to confirm selection
button = widgets.Button(description='Filter Rows')

# Output widget to display the filtered DataFrame
output = widgets.Output()

# Callback for filtering rows
def on_button_click(b):
    with output:
        output.clear_output()
        selected_indices = list(row_selector.value)
        global filtered_df  # Store filtered DataFrame globally
        filtered_df = df.iloc[selected_indices]
        print("Filtered DataFrame:")
        display(filtered_df)

# Attach callback
button.on_click(on_button_click)

# Display the widgets
display(row_selector, button, output)



SelectMultiple(description='Select Rows:', layout=Layout(width='50%'), options=(('MaxQuant_20241216_122433 (ha…

Button(description='Filter Rows', style=ButtonStyle())

Output()

In [5]:
filtered_df["intermediate_hash"]

0     00e2f863939301a2a71178652972dad895b27520
1     0280a06fabdbe84746419d0810deae56e7ab2406
2     0b2ce2ef84562945a7ad4865ff11ff720f8d68ad
3     0c88ca16d30d2911dc78febe087650427365e963
4     0d33cae7fd689507eb8b31c7ee2adc72adc3befe
5     1438823a0441d7a32c0d1095074cbc4a187ce2fb
6     1bfa914c771321b285a9ca40d4aa538cb9fdc42e
7     254d6c77ce656888918e738772ad5f5f6f1543e4
8     2805f6b6d1fabd65448540bee743f2b05c1dda59
9     2a5ef6191b8098757d490acf76bb9c2af5b89a39
10    36b7b01b380f641722b3b34633bb53d72348eb80
11    3bd27fcfd7013d81b578e850035874af14b1cc26
12    3cffb02abcf3ce5950dae07bb94985e12d632da9
13    4455fe37836eb6ae90f3592f214038c3a2b28b23
14    45486140efcbe205e2485f1ef4d668ec3d79fb99
15    45fbb2dc2468abd6dbaec4520c2a0e96cb7f7e0f
16    47db7ef37a0fb5fec79f3bedbfb4f67835774f10
17    4b50085fa82695a0d0b67283f022aff106c0a983
18    4decb9e0d2d9ed9f9be6a9ad5aa066b1dcd1e616
19    58dfec050e16a3ff10999c76d727f23a5cd6c6f9
20    5d7209cf235899ad053ed7dc6b839ac67a226b3c
21    5dc4492

In [6]:
# Step 1: Extract the hash list from the DataFrame
hash_list = filtered_df["intermediate_hash"].tolist()

# Step 2: Fetch folder names from the webpage
base_url = "https://proteobench.cubimed.rub.de/datasets/"
response = requests.get(base_url)
response.raise_for_status()  # Check for errors

soup = BeautifulSoup(response.text, "html.parser")
folder_links = [link['href'].strip("/") for link in soup.find_all("a") if link['href'].endswith("/")]

# Step 3: Filter folder links based on the hash list
matching_folders = [folder for folder in folder_links if folder in hash_list]

# Step 4: Download and extract zip files from matching folders
for folder in matching_folders:
    folder_url = f"{base_url}{folder}/"
    print(f"Processing folder: {folder_url}")

    # Fetch the folder page
    folder_response = requests.get(folder_url)
    folder_response.raise_for_status()

    folder_soup = BeautifulSoup(folder_response.text, "html.parser")
    zip_files = [link['href'] for link in folder_soup.find_all("a") if link['href'].endswith(".zip")]

    # Process each .zip file
    for zip_file in zip_files:
        zip_url = f"{folder_url}{zip_file}"
        print(f"Downloading: {zip_url}")

        # Download with a progress bar
        zip_response = requests.get(zip_url, stream=True)
        zip_response.raise_for_status()

        zip_filename = os.path.basename(zip_file)
        total_size = int(zip_response.headers.get('content-length', 0))
        block_size = 1024  # 1 KB

        # Save the zip file
        with open(zip_filename, "wb") as f, tqdm(
            desc=f"Downloading {zip_filename}",
            total=total_size,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
        ) as progress:
            for data in zip_response.iter_content(block_size):
                f.write(data)
                progress.update(len(data))

        # Extract the zip file
        extract_dir = f"extracted_files/{folder}"
        os.makedirs(extract_dir, exist_ok=True)
        with zipfile.ZipFile(zip_filename, "r") as zip_ref:
            zip_ref.extractall(extract_dir)
            print(f"Extracted contents to: {extract_dir}")

        # Cleanup downloaded .zip file
        os.remove(zip_filename)


Processing folder: https://proteobench.cubimed.rub.de/datasets/00e2f863939301a2a71178652972dad895b27520/
Downloading: https://proteobench.cubimed.rub.de/datasets/00e2f863939301a2a71178652972dad895b27520/00e2f863939301a2a71178652972dad895b27520_data.zip


Extracted contents to: extracted_files/00e2f863939301a2a71178652972dad895b27520
Processing folder: https://proteobench.cubimed.rub.de/datasets/0280a06fabdbe84746419d0810deae56e7ab2406/
Downloading: https://proteobench.cubimed.rub.de/datasets/0280a06fabdbe84746419d0810deae56e7ab2406/0280a06fabdbe84746419d0810deae56e7ab2406_data.zip


Extracted contents to: extracted_files/0280a06fabdbe84746419d0810deae56e7ab2406
Processing folder: https://proteobench.cubimed.rub.de/datasets/0c88ca16d30d2911dc78febe087650427365e963/
Downloading: https://proteobench.cubimed.rub.de/datasets/0c88ca16d30d2911dc78febe087650427365e963/0c88ca16d30d2911dc78febe087650427365e963_data.zip


Extracted contents to: extracted_files/0c88ca16d30d2911dc78febe087650427365e963
Processing folder: https://proteobench.cubimed.rub.de/datasets/0d33cae7fd689507eb8b31c7ee2adc72adc3befe/
Downloading: https://proteobench.cubimed.rub.de/datasets/0d33cae7fd689507eb8b31c7ee2adc72adc3befe/0d33cae7fd689507eb8b31c7ee2adc72adc3befe_data.zip


Extracted contents to: extracted_files/0d33cae7fd689507eb8b31c7ee2adc72adc3befe
Processing folder: https://proteobench.cubimed.rub.de/datasets/1438823a0441d7a32c0d1095074cbc4a187ce2fb/
Downloading: https://proteobench.cubimed.rub.de/datasets/1438823a0441d7a32c0d1095074cbc4a187ce2fb/1438823a0441d7a32c0d1095074cbc4a187ce2fb_data.zip


...

Extracted contents to: extracted_files/c9b6df263cfd4491f6188ac49043fa78d0f8acdb
Processing folder: https://proteobench.cubimed.rub.de/datasets/d8db9ffd261d4e4f5810738c726453dc034b3789/
Downloading: https://proteobench.cubimed.rub.de/datasets/d8db9ffd261d4e4f5810738c726453dc034b3789/d8db9ffd261d4e4f5810738c726453dc034b3789_data.zip


Extracted contents to: extracted_files/d8db9ffd261d4e4f5810738c726453dc034b3789
Processing folder: https://proteobench.cubimed.rub.de/datasets/ddae18c13fb1c8b8ab82231e8c09dc27df896829/
Downloading: https://proteobench.cubimed.rub.de/datasets/ddae18c13fb1c8b8ab82231e8c09dc27df896829/ddae18c13fb1c8b8ab82231e8c09dc27df896829_data.zip


Extracted contents to: extracted_files/ddae18c13fb1c8b8ab82231e8c09dc27df896829
Processing folder: https://proteobench.cubimed.rub.de/datasets/e0ab1339a5354cb27d895ff252383ee4a3365b2e/
Downloading: https://proteobench.cubimed.rub.de/datasets/e0ab1339a5354cb27d895ff252383ee4a3365b2e/e0ab1339a5354cb27d895ff252383ee4a3365b2e_data.zip


Extracted contents to: extracted_files/e0ab1339a5354cb27d895ff252383ee4a3365b2e
Processing folder: https://proteobench.cubimed.rub.de/datasets/e683472e109ece5b3e0996d8bef476ed0ad6dbdb/
Downloading: https://proteobench.cubimed.rub.de/datasets/e683472e109ece5b3e0996d8bef476ed0ad6dbdb/e683472e109ece5b3e0996d8bef476ed0ad6dbdb_data.zip


Extracted contents to: extracted_files/e683472e109ece5b3e0996d8bef476ed0ad6dbdb


In [7]:
token = toml.load("../webinterface/.streamlit/secrets.toml")["gh"]["token"]

# TODO change to the correct module
module_obj = DDAQuantIonModule(token=token)
results_df = module_obj.obtain_all_data_points(all_datapoints=None)

results_df.head(5)

,id,old_new,software_name,software_version,search_engine,search_engine_version,ident_fdr_psm,ident_fdr_peptide,ident_fdr_protein,enable_match_between_runs,...,hover_text,scatter_size,scan_window,quantification_method_DIANN,second_pass,protein_inference,predictors_library,quantification_method,mean_abs_epsilon,abundance_normalization_ions
0,MaxQuant_20241216_122433,old,MaxQuant,2.5.1.0,Andromeda,None,None,0.01,0.01,False,...,ProteoBench ID: MaxQuant_20241216_122433<br>So...,20.0,None,None,None,None,None,None,0.259947,None
1,MaxQuant_20241216_124040,old,MaxQuant,2.3.1.0,Andromeda,None,None,0.01,0.01,True,...,ProteoBench ID: MaxQuant_20241216_124040<br>So...,20.0,None,None,None,None,None,None,0.294623,None
2,MaxQuant_20250506_151623,old,MaxQuant,2.3.1.0,Andromeda,None,0.01,None,0.01,True,...,None,NaN,None,None,None,None,None,None,0.294623,None
3,FragPipe_20250504_164850,old,FragPipe,23.0,MSFragger,4.2,0.01,None,0.01,False,...,ProteoBench ID: FragPipe_20250504_164850<br>So...,20.0,None,None,None,ProteinProphet: --maxppmdiff 2000000,None,IonQuant,0.285003,1
4,WOMBAT_20250506_073548,old,WOMBAT,0.9.11,Proline,nan,0.01,0.01,0.01,True,...,ProteoBench ID: WOMBAT_20250506_073548<br>Soft...,40.0,None,None,None,nan,None,nan,0.326503,


In [8]:
from pathlib import Path

extra_path = Path("extracted_files")
submission_files = []

for idx, row in filtered_df.iterrows():
    base_path = extra_path / row["intermediate_hash"]

    if not base_path.exists():
        print(f"Warning: Base path {base_path} does not exist. Skipping.")
        continue

    # Read user comments
    comments_file = base_path / "comment.txt"
    if comments_file.exists():
        comments = "\n".join(open(comments_file).readlines())
    else:
        print(f"Warning: Comment file not found in {base_path}.")
        comments = ""

    # Search for input file starting with "input_file"
    input_files = list(base_path.glob("input_file*"))
    if input_files:
        input_file = input_files[0]  # Take the first match (or adjust selection logic)
    else:
        print(f"Warning: No input file starting with 'input_file' found in {base_path}.")
        input_file = None  # Or handle according to your application

    # Search for parameter file starting with "param_"
    param_files = list(base_path.glob("param_*"))
    if param_files:
        parameter_file = param_files[0]  # Take the first match
    else:
        print(f"Warning: No parameter file starting with 'param_' found in {base_path}.")
        parameter_file = None

    submission_files.append(
        {
            "input_file": input_file,
            "param_file": parameter_file,
            "input_type": row["software_name"],
            "default_cutoff_min_prec": 3,
            "user_comments": comments,
        }
    )

In [9]:
submission_files

[{'input_file': WindowsPath('extracted_files/00e2f863939301a2a71178652972dad895b27520/input_file.txt'),
  'param_file': WindowsPath('extracted_files/00e2f863939301a2a71178652972dad895b27520/param_0.txt'),
  'input_type': 'MaxQuant',
  'default_cutoff_min_prec': 3,
  'user_comments': ''},
 {'input_file': WindowsPath('extracted_files/0280a06fabdbe84746419d0810deae56e7ab2406/input_file.txt'),
  'param_file': WindowsPath('extracted_files/0280a06fabdbe84746419d0810deae56e7ab2406/param_0.txt'),
  'input_type': 'MaxQuant',
  'default_cutoff_min_prec': 3,
  'user_comments': ''},
 {'input_file': WindowsPath('extracted_files/0c88ca16d30d2911dc78febe087650427365e963/input_file.tsv'),
  'param_file': WindowsPath('extracted_files/0c88ca16d30d2911dc78febe087650427365e963/param_0..workflow'),
  'input_type': 'FragPipe',
  'default_cutoff_min_prec': 3,
  'user_comments': ''},
 {'input_file': WindowsPath('extracted_files/0d33cae7fd689507eb8b31c7ee2adc72adc3befe/input_file.csv'),
  'param_file': Windows

In [10]:
token = toml.load("../webinterface/.streamlit/secrets.toml")["gh"]["token"]

for submission_settings in submission_files:
    # TODO change to the correct module
    module_obj = DDAQuantIonModule(token=token)
    results_df = module_obj.obtain_all_data_points(all_datapoints=None)

    param_file = submission_settings["param_file"]
    input_file = submission_settings["input_file"]
    input_type = submission_settings["input_type"]
    default_cutoff_min_prec = submission_settings["default_cutoff_min_prec"]
    user_comments = submission_settings["user_comments"]

    user_config = defaultdict(lambda: "")

    results_intermediates, results_df_new, parsed_input = module_obj.benchmarking(
        input_file,
        input_type,
        user_config,
        results_df,
        default_cutoff_min_prec=default_cutoff_min_prec,
    )

    results_df_new.tail(5)

    try:
        param_obj = module_obj.load_params_file(
            [param_file], input_type
        )
    except:
        continue

    pr_url = module_obj.clone_pr(
        results_df_new,
        param_obj,
        remote_git="",
        submission_comments=user_comments,
    )

    print(f"Submitted: {submission_settings}")
    print("------------------------")

C:\Users\robbi\AppData\Local\Temp\tmpebe1w6ci
extracted_files\00e2f863939301a2a71178652972dad895b27520\param_0.txt extracted_files\00e2f863939301a2a71178652972dad895b27520\input_file.txt MaxQuant 3 


C:\Users\robbi\OneDrive\Documents\GitHub\ProteoBench\proteobench\io\parsing\parse_ion.py:31: UserWarning: 

                because they are implicit. Only after providing the appropriate parameter file,

                fixed modifications will be added correctly.
                
  warnings.warn(
Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main
INFO:github.Requester:Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main


Submitted: {'input_file': WindowsPath('extracted_files/00e2f863939301a2a71178652972dad895b27520/input_file.txt'), 'param_file': WindowsPath('extracted_files/00e2f863939301a2a71178652972dad895b27520/param_0.txt'), 'input_type': 'MaxQuant', 'default_cutoff_min_prec': 3, 'user_comments': ''}
------------------------
C:\Users\robbi\AppData\Local\Temp\tmpo9ktw2ni
extracted_files\0280a06fabdbe84746419d0810deae56e7ab2406\param_0.txt extracted_files\0280a06fabdbe84746419d0810deae56e7ab2406\input_file.txt MaxQuant 3 


C:\Users\robbi\OneDrive\Documents\GitHub\ProteoBench\proteobench\io\parsing\parse_ion.py:31: UserWarning: 

                because they are implicit. Only after providing the appropriate parameter file,

                fixed modifications will be added correctly.
                
  warnings.warn(
Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main
INFO:github.Requester:Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main


Submitted: {'input_file': WindowsPath('extracted_files/0280a06fabdbe84746419d0810deae56e7ab2406/input_file.txt'), 'param_file': WindowsPath('extracted_files/0280a06fabdbe84746419d0810deae56e7ab2406/param_0.txt'), 'input_type': 'MaxQuant', 'default_cutoff_min_prec': 3, 'user_comments': ''}
------------------------
C:\Users\robbi\AppData\Local\Temp\tmpgpdiym13
extracted_files\0c88ca16d30d2911dc78febe087650427365e963\param_0..workflow extracted_files\0c88ca16d30d2911dc78febe087650427365e963\input_file.tsv FragPipe 3 
C:\Users\robbi\AppData\Local\Temp\tmpvel2r80y
extracted_files\0d33cae7fd689507eb8b31c7ee2adc72adc3befe\param_0..yml extracted_files\0d33cae7fd689507eb8b31c7ee2adc72adc3befe\input_file.csv WOMBAT 3 Running Proline with lower precursor and fragment tolerance.

Load locally: extracted_files\0d33cae7fd689507eb8b31c7ee2adc72adc3befe\param_0..yml


...

Submitted: {'input_file': WindowsPath('extracted_files/ddae18c13fb1c8b8ab82231e8c09dc27df896829/input_file.tsv'), 'param_file': WindowsPath('extracted_files/ddae18c13fb1c8b8ab82231e8c09dc27df896829/param_0..json'), 'input_type': 'Sage', 'default_cutoff_min_prec': 3, 'user_comments': ''}
------------------------
C:\Users\robbi\AppData\Local\Temp\tmpx0115f1q
extracted_files\e0ab1339a5354cb27d895ff252383ee4a3365b2e\param_0.txt extracted_files\e0ab1339a5354cb27d895ff252383ee4a3365b2e\input_file.txt i2MassChroQ 3 
software_name i2MassChroQ
software_version 1.2.1
search_engine X!Tandem
search_engine_version nan
ident_fdr_psm 0.00969835888743
ident_fdr_peptide 0.0132251725326
ident_fdr_protein 0.00859598853868
enable_match_between_runs True
precursor_mass_tolerance [-10 ppm, 10 ppm]
fragment_mass_tolerance [-0.02 Da, 0.02 Da]
enzyme [RK]|{P}
allowed_miscleavages 2
min_peptide_length None
max_peptide_length None
fixed_mods 57.02146@C
variable_mods 15.99491@M;Acetyl(N-term);Pyrolidone(N-term)
m

Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main
INFO:github.Requester:Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main


Submitted: {'input_file': WindowsPath('extracted_files/e0ab1339a5354cb27d895ff252383ee4a3365b2e/input_file.txt'), 'param_file': WindowsPath('extracted_files/e0ab1339a5354cb27d895ff252383ee4a3365b2e/param_0.txt'), 'input_type': 'i2MassChroQ', 'default_cutoff_min_prec': 3, 'user_comments': ''}
------------------------
C:\Users\robbi\AppData\Local\Temp\tmpgphw4z_a
extracted_files\e683472e109ece5b3e0996d8bef476ed0ad6dbdb\param_0.txt extracted_files\e683472e109ece5b3e0996d8bef476ed0ad6dbdb\input_file.txt MaxQuant 3 


C:\Users\robbi\OneDrive\Documents\GitHub\ProteoBench\proteobench\io\parsing\parse_ion.py:31: UserWarning: 

                because they are implicit. Only after providing the appropriate parameter file,

                fixed modifications will be added correctly.
                
  warnings.warn(
Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main
INFO:github.Requester:Following Github server redirection from /repos/Proteobot/Results_quant_ion_DDA/branches/master to /repos/Proteobot/Results_quant_ion_DDA/branches/main


Submitted: {'input_file': WindowsPath('extracted_files/e683472e109ece5b3e0996d8bef476ed0ad6dbdb/input_file.txt'), 'param_file': WindowsPath('extracted_files/e683472e109ece5b3e0996d8bef476ed0ad6dbdb/param_0.txt'), 'input_type': 'MaxQuant', 'default_cutoff_min_prec': 3, 'user_comments': ''}
------------------------


In [11]:
import importlib
import proteobench.modules.quant.quant_base_module
import proteobench.modules.quant.quant_lfq_ion_DDA

importlib.reload(proteobench.modules.quant.quant_base_module)
importlib.reload(proteobench.modules.quant.quant_lfq_ion_DDA)

from proteobench.modules.quant.quant_lfq_ion_DDA import DDAQuantIonModule